In [1]:
import os,sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from plotnine import *
from statsmodels.stats.multitest import multipletests
from pathlib import Path
from tqdm.auto import tqdm
import re
import glob
from scipy import stats

In [6]:
def summarize_exp_stats(lane_path, lane_list):
    dfs = []  
    for lanes in lane_list:
        df = pd.read_csv(glob.glob(os.path.join(lane_path, lanes, "*_guide_count_info.csv"))[0])
        df.set_index('guide_id', inplace= True)
        dfs.append(df)
    num_cols = ['n_cells','sum_guide','sumsq_guide','ntc_cells','sum_ntc','sumsq_ntc']
    
    summary_ = pd.concat([df[num_cols] for df in dfs], axis=0).groupby('guide_id').sum()
    return summary_

In [2]:
def compute_stats_ko(df):
    pseudocount = 5e-2

    # Means
    df["guide_mean"] = df["sum_guide"] / df["n_cells"]
    df["ntc_mean"]   = df["sum_ntc"]   / df["ntc_cells"]

    # Variances (safe when n>=2)
    df["guide_var"] = (df["sumsq_guide"] - (df["sum_guide"]**2)/df["n_cells"]) / (df["n_cells"] - 1)
    df["ntc_var"]   = (df["sumsq_ntc"]   - (df["sum_ntc"]**2)/df["ntc_cells"]) / (df["ntc_cells"] - 1)

    df["guide_std"] = np.sqrt(df["guide_var"])
    df["ntc_std"]   = np.sqrt(df["ntc_var"])

    # Handle NaN/0 std
    df.fillna({'guide_std': 0, 'ntc_std': 0}, inplace=True)
    df.loc[:, "guide_std"] = np.where(df["guide_std"] == 0, 0.01, df["guide_std"])
    df.loc[:, "ntc_std"]   = np.where(df["ntc_std"]   == 0, 0.01, df["ntc_std"])

    # Decide direction
    
    # Fold-change definition (do once)
    df["fc"] = (df["ntc_mean"] + pseudocount) / (df["guide_mean"] + pseudocount)  # >1 means knockdown

    # Welch t-test per row
    t_stats, p_ones = [], []
    for _, row in df.iterrows():
        t_stat, p_two = stats.ttest_ind_from_stats(
            row["guide_mean"], row["guide_std"], row["n_cells"],
            row["ntc_mean"],   row["ntc_std"],   row["ntc_cells"],
            equal_var=False
        )
            
        if (t_stat < 0):
            p_one = (p_two / 2) 
        else:
            p_one = 1.0

        t_stats.append(t_stat)
        p_ones.append(p_one)

    df["t_statistic"] = t_stats
    df["p_value"] = p_ones

    # FDR
    mask = df["p_value"].notna()
    df["adj_pvals"] = np.nan
    df.loc[mask, "adj_pvals"] = multipletests(df.loc[mask, "p_value"], method="fdr_bh")[1]

    # Cohen's d (pooled SD)
    n1 = df["n_cells"].astype(float)
    n2 = df["ntc_cells"].astype(float)
    sp2 = ((n1 - 1)*df["guide_var"] + (n2 - 1)*df["ntc_var"]) / (n1 + n2 - 2)
    sp  = np.sqrt(sp2)

    df["effect_size"] = np.nan
    m = (n1 >= 2) & (n2 >= 2) & (sp > 0)
    df.loc[m, "effect_size"] = (df.loc[m, "guide_mean"] - df.loc[m, "ntc_mean"]) / sp[m]

    return df

In [3]:
dir_list = pd.read_csv('/Users/chandrima.modak/Gladstone Dropbox/Chandrima Modak/macs_perturbseq_analysis/experiment_info.csv')
lane_cols_for_eff = [c for c in dir_list.columns]
run = []
for _, row in dir_list[lane_cols_for_eff].iterrows():
    for col in lane_cols_for_eff:
       run.append(f'{row[col]}_{col}')

In [4]:
lane_path = '/Users/chandrima.modak/Gladstone Dropbox/Chandrima Modak/macs_perturbseq_analysis/'
ctrl_list = []
lps_list = []
for i in run:
    if 'Ctrl' in i:
        ctrl_list.append(i)
    else:
        lps_list.append(i)

In [7]:
df_ctrl =  summarize_exp_stats(lane_path, ctrl_list)
df_ctrl

,n_cells,sum_guide,sumsq_guide,ntc_cells,sum_ntc,sumsq_ntc
guide_id,,,,,,
A4GALT-1,578,17.812806,15.402874,29628,564.380737,477.377357
A4GALT-2,390,13.338955,10.076600,29628,564.380737,477.377357
A4GALT-3,827,23.268857,18.793370,29628,564.380737,477.377357
A4GALT-4,951,19.837996,16.570582,29628,564.380737,477.377357
AAGAB-1,844,369.662735,363.413429,29628,17444.345459,18807.167725
...,...,...,...,...,...,...
ZNF79-4,645,19.992992,13.534191,29628,1139.320648,881.889221
ZNF823-1,750,47.474886,39.372021,29628,2265.794250,1867.140106
ZNF823-2,719,65.517632,47.691523,29628,2265.794250,1867.140106


In [8]:
df_lps = summarize_exp_stats(lane_path, lps_list)
df_lps

,n_cells,sum_guide,sumsq_guide,ntc_cells,sum_ntc,sumsq_ntc
guide_id,,,,,,
A4GALT-1,684,129.025307,121.841091,33146,7415.184814,7427.030273
A4GALT-2,416,83.572794,83.632089,33146,7415.184814,7427.030273
A4GALT-3,1060,248.192352,243.806694,33146,7415.184814,7427.030273
A4GALT-4,1135,225.813587,219.349014,33146,7415.184814,7427.030273
AAGAB-1,893,437.408257,450.637596,33146,21135.842285,23689.932129
...,...,...,...,...,...,...
ZNF79-4,818,19.905251,14.135347,33146,1104.591644,864.881454
ZNF823-1,910,33.340771,27.321822,33146,1333.877625,1084.952240
ZNF823-2,905,45.083932,37.499981,33146,1333.877625,1084.952240


In [9]:
path_fdr = '/Users/chandrima.modak/Gladstone Dropbox/Chandrima Modak/macs_perturbseq_analysis/qc_stats'
df_ctrl_fdr = compute_stats_ko(df_ctrl)
df_lps_fdr = compute_stats_ko(df_lps)

# Cell level guide efficiency

In [10]:
df_ctrl = []
df_lps = []
for i in run:
    cell_ko_pattern = glob.glob(os.path.join('/Users/chandrima.modak/Gladstone Dropbox/Chandrima Modak/macs_perturbseq_analysis', i, '*cells_based_guide_efficiency.csv'))[0]
    df = pd.read_csv(cell_ko_pattern)
    if 'Ctrl' in i:
        df_ctrl.append(df)
    else:
        df_lps.append(df)

In [11]:
dfs_ctrl = pd.concat(df_ctrl,axis=0)
dfs_lps = pd.concat(df_lps,axis=0)
dfs_ctrl['signif_ko_95perc'] = dfs_ctrl['ntc_fraction'] < 0.05
dfs_lps['signif_ko_95perc'] = dfs_lps['ntc_fraction'] < 0.05

In [12]:
def ko_cells_percent(df):
    n_kos = df.groupby('guide_id')['signif_ko_95perc'].sum()
    n_tot = df.groupby('guide_id').size()
    df_ko = pd.DataFrame(n_kos/n_tot, columns= ['percent_cells_signi_ko'])
    df_ko
    return df_ko

In [13]:
df_ctrl_ko = ko_cells_percent(dfs_ctrl)
df_lps_ko = ko_cells_percent(dfs_lps)

In [14]:
df_ctrl_fdr =pd.merge(df_ctrl_fdr, df_ctrl_ko, on = 'guide_id', how= 'inner')
df_lps_fdr = pd.merge(df_lps_fdr, df_lps_ko, on = 'guide_id', how= 'inner')

In [15]:
df_ctrl_fdr

,n_cells,sum_guide,sumsq_guide,ntc_cells,sum_ntc,sumsq_ntc,guide_mean,ntc_mean,guide_var,ntc_var,guide_std,ntc_std,fc,t_statistic,p_value,adj_pvals,effect_size,percent_cells_signi_ko
guide_id,,,,,,,,,,,,,,,,,,
A4GALT-1,578,17.812806,15.402874,29628,564.380737,477.377357,0.030818,0.019049,0.025743,0.015750,0.160447,0.125499,0.854375,1.753066,1.000000e+00,1.000000e+00,0.093215,0.958478
A4GALT-2,390,13.338955,10.076600,29628,564.380737,477.377357,0.034202,0.019049,0.024731,0.015750,0.157261,0.125499,0.820034,1.895015,1.000000e+00,1.000000e+00,0.120303,0.948718
A4GALT-3,827,23.268857,18.793370,29628,564.380737,477.377357,0.028136,0.019049,0.021960,0.015750,0.148188,0.125499,0.883696,1.746158,1.000000e+00,1.000000e+00,0.072027,0.961306
A4GALT-4,951,19.837996,16.570582,29628,564.380737,477.377357,0.020860,0.019049,0.017007,0.015750,0.130411,0.125499,0.974439,0.422077,1.000000e+00,1.000000e+00,0.014414,0.972660
AAGAB-1,844,369.662735,363.413429,29628,17444.345459,18807.167725,0.437989,0.588779,0.239033,0.288126,0.488910,0.536773,1.309003,-8.810152,3.160953e-18,1.579748e-17,-0.281584,0.490521
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZNF79-4,645,19.992992,13.534191,29628,1139.320648,881.889221,0.030997,0.038454,0.020054,0.028288,0.141610,0.168189,1.092069,-1.317341,9.408275e-02,1.469154e-01,-0.044477,0.950388
ZNF823-1,750,47.474886,39.372021,29628,2265.794250,1867.140106,0.063300,0.076475,0.048554,0.057173,0.220349,0.239109,1.116284,-1.613572,5.350874e-02,8.687160e-02,-0.055203,0.912000
ZNF823-2,719,65.517632,47.691523,29628,2265.794250,1867.140106,0.091123,0.076475,0.058108,0.057173,0.241055,0.239109,0.896201,1.610338,1.000000e+00,1.000000e+00,0.061251,0.862309


In [16]:
df_ctrl_fdr.to_csv(os.path.join(path_fdr, 'Macs_Ctrl_cell_level_guide_efficiency.csv'))
df_lps_fdr.to_csv(os.path.join(path_fdr, 'Macs_LPS_cell_level_guide_efficiency.csv'))